In [247]:
# 공통
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
import datetime
import matplotlib.pyplot as plt

In [268]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체1.csv', encoding='cp949')

# data = data.drop(['소비 금액'], axis = 1)
data = data.drop(['평균온도'], axis = 1)
data = data.drop(['평균습도'], axis = 1)
# data = data.drop(['이용 횟수'], axis = 1)

data = data[:8760]
data = data.set_index('일자')
data = data.fillna(method='ffill')
data

,소비 금액,이용 횟수
일자,,
2021-01-01 0:00,2683380.0,72.0
2021-01-01 1:00,499360.0,11.0
2021-01-01 2:00,64176.0,10.0
2021-01-01 3:00,885833.0,11.0
2021-01-01 4:00,0.0,0.0
...,...,...
2021-12-31 19:00,5136390.0,63.0
2021-12-31 20:00,3124749.0,72.0
2021-12-31 21:00,1210011.0,33.0


In [249]:
# 2021년 예측

# train = data[:8016]['소비 금액']

# exog1 = data.drop('소비 금액', axis=1)[:8016]
# exog1 = sm.add_constant(exog1)

# exog2 = data.drop('소비 금액', axis=1)[8016:]
# exog2 = sm.add_constant(exog2)

In [250]:
# 2022년 예측

train = data[:'2021-12-31 23:00']['소비 금액']

exog1 = data.drop('소비 금액', axis=1)[:'2021-12-31 23:00']

exog2 = data.drop('소비 금액', axis=1)['2022-01-01 0:00':]

In [251]:
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
df_list = [start + relativedelta(hours = x) for x in range(0, len(data)) ]

# # 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기) p, q
# import itertools

# # Define the p, d and q parameters to take any value between 0 and 2
# p = d = q = range(0, 2)

# # Generate all different combinations of p, q and d triplets
# pdq = list( itertools.product(p, d, q))

# # Generate all different combinations of seasonal p, q and d triplets
# seasonal_pdq = [ (x[0], x[1], x[2], 24)  for x in pdq ]

# print('Example of parameter combinations for Seasonal ARIMA ...')
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[1]) )
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[2]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[3]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[4]) )

In [252]:
# # 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기) p, q
# import itertools

# p = d = q = range(0, 2)
# pdq = list( itertools.product(p, d, q))
# seasonal_pdq = [ (x[0], x[1], x[2], 24)  for x in pdq ]

# select_candi = 10000000
# param_candi = ( 0, 0, 0 )
# param_seasonal_candi = ( 0, 0, 0)

# count=0
# end_count = len(pdq)

# for param in pdq:   
#     for param_seasonal in seasonal_pdq:
#         try:
#             mod = sm.tsa.statespace.SARIMAX(data['소비 금액'],
#                                             order=param,
#                                             seasonal_order=param_seasonal,
#                                             enforce_stationarity=False,
#                                             enforce_invertibility=False
#                                            )
#             results = mod.fit()
#             count += 1
#             if count <= 5:
#                 print('SARIMA{}x{}24 - AIC:{}'.format(param, param_seasonal, results.aic))
            
#             if results.aic < select_candi:
#                 select_candi = results.aic
#                 param_candi = param
#                 param_seasonal_candi = param_seasonal
#         except:
#             continue
            
# print(param_candi, param_seasonal_candi, select_candi) 
# print('Order : {}, Seasonal_Order : {}'.format(param_candi, param_seasonal_candi))
# print('Order : {}, Seasonal_Order : {}, AIC : {}'.format(param_candi, param_seasonal_candi, select_candi))

In [253]:
# # Order : (1, 1, 1), Seasonal_Order : (1, 0, 1, 24) 2057135.3018430097
# # order = (0, 0, 0), seasonal_order = (1, 1, 1, 24) 2061582.8357607473

# def predict(exog1, exog2):

#     mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, 
#                             order = (1, 1, 1), seasonal_order = (1, 0, 1, 24))
#     fit_res = mod.fit(disp = False)

#     forecast = fit_res.forecast(steps = 744, exog = exog2)

#     # 2021년 실제값
#     actual = data['소비 금액'][8016:]

#     rmse = np.sqrt(mean_squared_error(actual, forecast))
    
#     return rmse

# predict(exog1, exog2)

# # 변수없이 12032081.903667446 / 12822899.653348675
# # 전체평균 11134711.02414859 / 14996102.978741448
# # 전체평균제외 13945093.033749819 / 15015795.613447221
# # 온도만 13141867.131440545 / 13840825.71410531
# # 습도만 12218263.756765371 / 13670886.534086179

In [254]:
# predict(exog1, exog2)

# # order = (1,0,0), seasonal_order = (2,0,1,7) / order = (1, 0, 1), seasonal_order = (0, 1, 1, 7)

# # 새로운 arima 코드.ipynb / 찬재.ipynb

# # 변수없이 12032081.903667446 / 12822899.653348675
# # 전체평균 11134711.02414859 / 14996102.978741448
# # 전체평균제외 13945093.033749819 / 15015795.613447221
# # 온도만 13141867.131440545 / 13840825.71410531
# # 습도만 12218263.756765371 / 13670886.534086179

In [255]:
mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, 
                        order = (1, 1, 1), seasonal_order = (1, 0, 1, 24))

fit_res = mod.fit(disp = False)
forecast = fit_res.forecast(steps = 8760, exog = exog2)

print(forecast)

#8760

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


2022-01-01 00:00:00    1.136838e+06
2022-01-01 01:00:00    6.062517e+05
2022-01-01 02:00:00    3.086159e+05
2022-01-01 03:00:00    1.838566e+05
2022-01-01 04:00:00    1.500776e+05
                           ...     
2022-12-31 19:00:00    3.618905e+06
2022-12-31 20:00:00    3.022522e+06
2022-12-31 21:00:00    2.543483e+06
2022-12-31 22:00:00    2.096278e+06
2022-12-31 23:00:00    1.478931e+06
Freq: H, Name: predicted_mean, Length: 8760, dtype: float64

In [258]:
data = pd.DataFrame(forecast, columns = ['일자', '2022년 소비 금액 예측 값'])
data

2022-01-01 00:00:00    1.136838e+06
2022-01-01 01:00:00    6.062517e+05
2022-01-01 02:00:00    3.086159e+05
2022-01-01 03:00:00    1.838566e+05
2022-01-01 04:00:00    1.500776e+05
                           ...     
2022-12-31 19:00:00    3.618905e+06
2022-12-31 20:00:00    3.022522e+06
2022-12-31 21:00:00    2.543483e+06
2022-12-31 22:00:00    2.096278e+06
2022-12-31 23:00:00    1.478931e+06
Freq: H, Name: predicted_mean, Length: 8760, dtype: float64


In [265]:
forecast.to_csv('소비금액만.csv', index= True)

In [267]:
df = pd.read_csv('소비금액만.csv')
df.columns = ['일자', '소비 금액']
df

,일자,소비 금액
0,2022-01-01 0:00,1136838
1,2022-01-01 1:00,606252
2,2022-01-01 2:00,308616
3,2022-01-01 3:00,183857
4,2022-01-01 4:00,150078
...,...,...
8755,2022-12-31 19:00,3618905
8756,2022-12-31 20:00,3022522
8757,2022-12-31 21:00,2543483
8758,2022-12-31 22:00,2096278


In [ ]:
# a = ['변수X', '모든 변수', '이용횟수', '평균온도', '평균습도', '평균온도, 평균습도', '이용횟수, 평균온도', '이용횟수, 평균습도']
# b = [2057135, 1953881, 1950687, 2859788, 3157025, 2530657, 2056692, 1952215]

# data = pd.DataFrame(zip(a, b), columns = ['변수', 'RMSE'])
# data